# Hamiltonian Simulation Problem Submission
This notebook contains the code to generate my submission for the Classiq Coding Competition's Hamiltonian Simulation Problem.

### Disclaimer
The goal of this notebook is to generate a minimal quantum circuit. It does not in any way aim to do so in a classical efficient way, and a lot of the algorithms used are far from optimized. Since the challenge doesn't seem to care much about the circuit-generating code, I didn't care much either.

## The Challenge
We are given a 10-qubit Hamiltonian that is represented as a linear combination of kronecker products of Pauli operators (276 unique operators in total). The goal of the challenge is to build a quantum circuit with minimal depth to simulate this Hamiltonian under the following constraints:
* Use no more than 10 qubits.
* Use only single-qubit gates and CNOT gates.
* Approximate the Hamiltonian to within 0.1 in the matrix 2-norm (largest singular value norm).
More details can be fund in the Classiq website: https://www.classiq.io/competition/hamiltonian#competitionForm

## The Approach
My approach to this challenge is heavily based around the following article:
[Berg, Ewout van den, and Kristan Temme. “Circuit Optimization of Hamiltonian Simulation by Simultaneous Diagonalization of Pauli Clusters.” Quantum, vol. 4, Sept. 2020, p. 322. arXiv.org,](https://doi.org/10.22331/q-2020-09-12-322).
The obvious approach, which is also hinted in the challenge's introduction by Classiq, is to implement circuits for the Pauli products and use Trotterization to approximate the full $e^{-iH}$ matrix. Trotterization will add a lot of depth to our circuit, so to minimize this depth we need to:
1. Minimize the ammount of Trotter iterations needed to reach at the wanted accuracy.
2. Reduce the depth of the circuit implementation of each Pauli product.

In [1]:
import numpy as np
from numpy import logical_xor as lx, logical_and as la
from scipy.linalg import expm

from qiskit.quantum_info import Pauli, Operator
from qiskit import QuantumCircuit, QuantumRegister

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Step 0
For startes, we need to read the Pauli strings provided by Classiq in their [blog post](https://quantum.discoursehosting.net/t/lih-hamiltonian/30), and use them to build the Hamiltonian matrix and its exponent, so we know what we need to approximate.

In [2]:
with open('LiH_hamiltonian.txt', 'r') as pauli_strings:
    paulis = [((-1 if sign == '-' else 1) * float(prefactor) * Pauli(s).to_matrix()) for sign, prefactor, _, s in 
          [pauli.strip().split(' ') for pauli in pauli_strings.read().splitlines() if pauli]]
H = sum(paulis)
eiH = expm(-1j * H)

### Step 1 - Grouping Commuting Paulis
Since exponents of commuting matrices commute, we don't need to apply troterrization between commuting Paulis. Therefore, we can group the given Paulis into mutually-commuting groups. We can then implelemnt a circuit for each Pauli in a group and compose them to implement the entire group EXACTLY, since no trotterization is needed within the groups. This approach should greatly reduce the ammount of trotterization needed, and therefore the circuit depth.

In [3]:
with open('LiH_hamiltonian.txt', 'r') as pauli_strings:
    paulis = [((-1 if sign == '-' else 1) * float(prefactor), Pauli(s)) for sign, prefactor, _, s in 
          [pauli.strip().split(' ') for pauli in pauli_strings.read().splitlines() if pauli]]

# step 1: partition the hamiltonian into mutually commuting subsets
commuting_subsets = []
for (t, pauli) in paulis:
    for subset in commuting_subsets:
        if all([pauli.commutes(p) for _, p in subset]):
            subset.append((t, pauli))
            break
    else:
        commuting_subsets.append([(t, pauli)])

### Step 2 - Simultaneous Diagonalization
Since the Paulis in each group mutually commute, there exists a basis that simultaneously diagnolizes the entire group, turning it into kronecker products of identity matrices and Pauli-Zs. Since the implementation of Pauli-X and Pauli-Y operators basically involve diagnolization and Pauli-Z, we can reduce the circuit depth of eah group by diagonalizing all of its members at the same time instead of doing so seperately for each one.

The algorithm to build a circuit that diagonalizes commuting Paulis is rather involved, and is detailed in the article linked above. This is mearly a python implementation of one of the suggested algorithms found in the article. I did, however, disagree with one of the notions in the article and had to work around it by adding SWAP gates (implemented using CNOT gates as per the challenge constraints).

To ensure a minimal circuit, I've added a small routine at the end of the algorithm to remove neighbouring Hadamard gates that cancel out, since the algorithm tends to produce them.

Overall, this step takes a set of commuting Paulis and returns a diagonalizing quantum circuit (containing only CNOT, Hadamard and S gates), and a new set of Paulis consisting of only Identity and Pauli-Z products. Applying the diagnoalizing circuit, the new Paulis, and the inverse of the diagnoalizing circuit, should have the same effect as implementing each Pauli and composing them.

In [4]:
# step 2: create a circuit to simultaneously diagonalize commuting paulis
def sim_cliff_diag(paulis):
    q = QuantumRegister(paulis[0].num_qubits, 'q')
    qc = QuantumCircuit(q)
    
    # swapping rows is equivalent to switching the order of the paulis
    
    # despite what the article says, I believe swapping columns does add a SWAP gate,
    # since it doesn't commute with the cliffords
    
    # sweeping the rows does not add a gate to the circuit.
    # It is equivalent to multiplying the operators.
    # If we find a basis that diagnoalizes both the product of two operators and one of the initial operators,
    # it will diagnoalize both original operators.
    # Therefore, sweeping rows also doesn't add gates to the circuit.
    def sweep_rows(X, Z, S, i, j):
        # the sign manipulation is redundant since we don't apply this operation to the actual circuit anyway,
        # but its here for completeness
        def g(x1, z1, x2, z2):
            if not x1 and not z1:
                return 0
            if x1 and z1:
                return z2 - x2
            if x1 and not z1:
                return z2 * (2*x2 - 1)
            if z1 and not x1:
                return x2 * (1 - 2*z2)
            
        t = 2*int(S[j][0]) + 2*int(S[i][0]) + sum([g(int(X[i,k]), int(Z[i,k]), int(X[j,k]), int(Z[j,k]))
                                                    for k in range(X.shape[1])])
        
        if t % 4 == 0:
            S[j][0] = False
        elif t % 4 == 2:
            S[j][0] = True
        else:
            raise Exception("Unexpected result in sweep_rows")
            
        X[[j], :] = lx(X[[j], :], X[[i], :])
        Z[[j], :] = lx(Z[[j], :], Z[[i], :])
            
    
    # The Clifford gates: CNOT, CZ, H and S, are useful for manipulating the matrix.
    # Unlike row/column swaps and row sweeps, they involve adding gates to the circuit.
    def h(X, Z, S, i):
        S[:,[0]] = lx(S[:,[0]], la(X[:,[i]], Z[:,[i]]))
        X[:,[i]], Z[:,[i]] = Z[:,[i]], X[:, [i]]
    
    def s(X, Z, S, i):
        S[:,[0]] = lx(S[:,[0]], la(X[:,[i]], Z[:,[i]]))
        Z[:,[i]] = lx(Z[:,[i]], X[:,[i]])
    
    def cnot(X, Z, S, i, j):
        Xb_Za_1 = lx(lx(X[:,[j]], Z[:,[i]]), np.array([[True] for _ in range(S.shape[0])]))
        S[:,[0]] = lx(S[:,[0]], la(Xb_Za_1, la(X[:,[i]], Z[:,[j]])))
        X[:,[j]] = lx(X[:,[j]], X[:,[i]])
        Z[:,[i]] = lx(Z[:,[i]], Z[:,[j]])
    
    def cz(X, Z, S, i, j):
        h(X, Z, S, j)
        cnot(X, Z, S, i, j)
        h(X, Z, S, j)
    
    # build interim simplectic matrix
    X = np.array([p.x for p in paulis])
    Z = np.array([p.z for p in paulis])
    S = np.array([[False] for _ in paulis])
    
    # create copies to hold the new paulis
    X2 = X.copy()
    Z2 = Z.copy()
    S2 = S.copy()
    
    r = np.linalg.matrix_rank(np.concatenate((X, Z), axis=1))

    # diagonalize X
    # these operations do not add gates to the circuit and therefore are only applied to the interim matrix
    kx = 0
    for k in range(r):
        for i in range(k, X.shape[0]):
            for j in range(k, X.shape[1]):
                if X[i,j]:
                    kx = k+1
                    if i != k:
                        # swap rows
                        X[[i, k], :] = X[[k, i], :]
                        Z[[i, k], :] = Z[[k, i], :]
                        S[[i, k], :] = S[[k, i], :]
                    if j != k:
                        # swap columns
                        X[:, [j, k]] = X[:, [k, j]]
                        Z[:, [j, k]] = Z[:, [k, j]]
                        X2[:, [j, k]] = X2[:, [k, j]]
                        Z2[:, [j, k]] = Z2[:, [k, j]]
                        # SWAP with CNOTs
                        qc.cnot(j, k)
                        qc.cnot(k, j)
                        qc.cnot(j, k)

                    for l in range(X.shape[0]):
                        if l == k:
                            continue
                        if X[l, k]:
                            # sweep rows
                            sweep_rows(X, Z, S, k, l)
                    break
            else:
                continue
            break
    
    # diagonalize Z
    # these operations do not add gates to the circuit and therefore are only applied to the interim matrix
    kz = kx
    for k in range(kx, r):
        for i in range(k, Z.shape[0]):
            for j in range(k, Z.shape[1]):
                if Z[i,j]:
                    kz = k+1
                    if i != k:
                        # swap rows
                        X[[i, k], :] = X[[k, i], :]
                        Z[[i, k], :] = Z[[k, i], :]
                        S[[i, k], :] = S[[k, i], :]
                    if j != k:
                        # swap columns
                        X[:, [j, k]] = X[:, [k, j]]
                        Z[:, [j, k]] = Z[:, [k, j]]
                        X2[:, [j, k]] = X2[:, [k, j]]
                        Z2[:, [j, k]] = Z2[:, [k, j]]
                        # SWAP with CNOTs
                        qc.cnot(j, k)
                        qc.cnot(k, j)
                        qc.cnot(j, k)
                        
                    for l in range(Z.shape[0]):
                        if l == k:
                            continue
                        if Z[l, k]:
                            # sweep rows
                            sweep_rows(X, Z, S, k, l)
                    break
            else:
                continue
            break

    # swap diag Z to X
    for j in range(kx, kz):
        # H(j)
        h(X, Z, S, j)
        h(X2, Z2, S2, j)
        qc.h(j)

    # eliminate off-diag X
    for i in range(kz):
        for j in range(kz, X.shape[1]):
            if X[i,j]:
                cnot(X, Z, S, i, j)
                cnot(X2, Z2, S2, i, j)
                qc.cnot(i, j)
    
    # eliminate off-diag Z
    for i in range(1, kz):
        for j in range(i):
            if Z[i, j]:
                # cz(i,j)
                cz(X, Z, S, i, j)
                cz(X2, Z2, S2, i, j)
                qc.h(j)
                qc.cnot(i, j)
                qc.h(j)
    
    for i in range(kz):
        # eliminate diag Z - so Z is all zeros
        if Z[i, i]:
            # S(i)
            s(X, Z, S, i)
            s(X2, Z2, S2, i)
            qc.s(i)
        
        # switch X and Z
        # H(i)
        h(X, Z, S, i)
        h(X2, Z2, S2, i)
        qc.h(i)

    # cleanup
    # remove redundant hadamards
    last_h = {i: None for i in range(q.size)}
    h_to_remove = []
    for i, (g, qubits, _) in enumerate(qc.data):
        if g.name == 'h':
            if last_h[qubits[0].index] is not None:
                h_to_remove.append(last_h[qubits[0].index])
                h_to_remove.append(i)
            else:
                last_h[qubits[0].index] = i
        else:
            for qubit in qubits:
                last_h[qubit.index] = None
    
    h_to_remove.sort()
    for i in h_to_remove[::-1]:
        qc.data.pop(i)

    # build new paulis from simplectic matrix
    assert np.logical_not(X).all(), "Failed Clifford diagonalization"
    assert np.logical_not(X2).all(), "Diagnolization worked in theory only"
    return qc, [Pauli((Z2[i], X2[i], 2 if S2[i] else 0)) for i in range(X2.shape[0])]

### Step 3 - Circuits for the Diagonal Paulis
Now we only need to build circuits to run the diagnoal Paulis we got after the previous step. Implementing these Paulis requires only CNOT and RZ gates (except for the global phase Pauli, which requires NOT and Phase gates). To further reduce the circuit depth, we wish to implement a circuit with a minimal amount of CNOT gates.

To do so, we try to reorder the diagonal Paulis in such a way that each Pauli is "close" to its neighbours, where being "close" means needing a minimal amount of CNOTs. We implement an recursive algorithm to apply minimal changes to the last Pauli we executed and search for a next Pauli that matches these changes.

Implementing the Pauli-Z products requires CNOTing the qubits and rotating one of them. I chose to always rotate the qubit with the lowest possible index, and CNOT the other qubits to it. This might not be the absolute optimal approach, but it was simple and good enough for me.

finally, after handling all Paulis that involve some Pauli-Z, we turn to implement the ones that only involve identitiy matrices, AKA global phases.

In [5]:
# step 3: reorder the new paulis to reduce the cnot count
# we'll do it using a recursion
# in each step, try to find paulis that are only 1 cnot away from the last pauli that was executed
def walk_paulis(diag_paulis):
    num_qubits = diag_paulis[0][1].num_qubits
    paulis_qc = QuantumCircuit(QuantumRegister(num_qubits, 'q'))
    perms = set()

    def walk(start):
        visited = []

        if 'Z' in start:
            rotating_qubit = start[::-1].find('Z')
            if len(start) == num_qubits:
                for i, (t, p) in enumerate(diag_paulis):
                    if p.to_label() == start:
                        paulis_qc.rz(2*t, rotating_qubit)
                        visited.append(i)
            
            else:
                visited = walk('I' + start)
                paulis_qc.cnot(len(start), rotating_qubit)
                visited_after_cnot = walk('Z' + start)
                if not visited_after_cnot:
                    paulis_qc.data.pop()
                else:
                    paulis_qc.cnot(len(start), rotating_qubit)
                    visited += visited_after_cnot
                    
        elif len(start) < num_qubits:
            visited = walk('I' + start)
            visited += walk('Z' + start)
        
        return visited
                
    # do all non-trivial paulis
    walk('')
    
    # do trivial paulis
    global_phase = sum([t for t, p in diag_paulis if p.to_label() == 'I' * num_qubits])
    if global_phase:
        paulis_qc.p(-global_phase, 0)
        paulis_qc.x(0)
        paulis_qc.p(-global_phase, 0)
        paulis_qc.x(0)

    return paulis_qc

### Step 4 - Trotterize
This step is straightforward at this point:
1. Use Step 2 to diagonalize each commuting group.
2. Use Step 3 to implement the diagnoal Paulis.
3. Sandwich the circuit from the previous step with the diagnoalizing circuit and its inverse, to implement each group.
4. Apply minimal Trotterization between the resuting circuits to arrive at the wanted accuracy.

Obviously, we want to apply as little Trotterization as possible, since the depth of our final circuit is proportional to the number of trotterization steps. For any given number of Trotterization steps $n$, each ordering of the commuting groups results in a different approximation error (unfortunately, in a way I can't predict). By trying random orders in a loop I was lucky enough to find an ordering that produces a small enough error after just 2 steps (surprisingly, this took less than 2 minutes, making me believe its not that rare).
I tried to find such an ordering for just 1 step, and managed to get as low as 0.15, but gave up after 2 hours (There are $21!$ possible orderings, each one takes about 3 seconds to check on my laptop, and I didn't have 4,860,249,445,558 years for this challenge).

In [6]:
# step 4 - troterrize
n = 2
circuits = []
for i, paulis in enumerate(commuting_subsets):
    diag_qc, diag_paulis = sim_cliff_diag([p for _, p in paulis])
    new_paulis = [((t/n) * (1 if p.phase == 0 else -1), Pauli((p.z, p.x, 0))) for (t, _), p in zip(paulis, diag_paulis)]
    diag_paulis_qc = walk_paulis(new_paulis)
    circuits.append(diag_qc.compose(diag_paulis_qc).compose(diag_qc.inverse()))

master_circuit = QuantumCircuit(QuantumRegister(len(circuits[0].qubits), 'q'))
for _ in range(n):
    for i in [6, 7, 9, 1, 11, 18, 4, 13, 12, 0, 19, 15, 17, 3, 5, 2, 20, 10, 8, 16, 14]:
        master_circuit = master_circuit.compose(circuits[i])

In [7]:
master_circuit.qasm(filename='LiH_hamiltonian.qasm')
o = Operator(master_circuit)
print(f'Approximation error: {np.linalg.norm(eiH - o.data, 2)}')
print(f'Circuit depth: {master_circuit.depth()}')

Approximation error: 0.08143930743852341
Circuit depth: 3193


## Results
All in all the produced circuit has a depth of 3193 (that's about 11 gates for each Pauli product in the original Hamiltonian) and gives an approximation error of about 0.08. Had the  threshold been 0.15, we could possibly find an ordering of the groups that would save a Trotter step, cutting the circuit depth in half.